## (a) Suppose we constructed our model using this configuration. How many trainable parameters
would our model have? Assuming each parameter is represented using single-precision floating
point, how much memory is required to just load this model?

Deliverable: A one-to-two sentence response.



### Counting parameters
Input Embedding: vocab_size x d_model


Transformer block:
- rms norm -> d_model
- q k v o proj weights-> d_model * d_model * 4
- rms norm -> d_model
- ffn w1 w2 w3 -> d_ff * d_model * 3

Transformer block total: (d_model + d_model * d_model * 4 + d_model + d_ff * d_model * 3)

rms norm final -> d_model

output linear -> d_model * vocab_size

softmax -> 0



In [1]:
# (a) Consider GPT-2 XL, which has the following configuration:
vocab_size = 50257
context_length = 1024
num_layers = 48
d_model = 1600

num_heads = 25
d_ff = 6400

In [ ]:
total_params = (
    vocab_size * d_model
    + num_layers * (d_model + d_model * d_model * 4 + d_model + d_ff * d_model * 3)
    + d_model
    + d_model * vocab_size
)

print(total_params)

1635537600



## (b) Identify the matrix multiplies required to complete a forward pass of our GPT-2 XL-shaped
model. How many FLOPs do these matrix multiplies require in total? Assume that our input
sequence has context_length tokens.

**Deliverable**: A list of matrix multiplies (with descriptions), and the total number of FLOPs
required.

### Input embedding

None 


### Transformer Block 

1. RMS Norm:

None

2. MHSA w/ RoPE

Projection: q k v o so 4 times


In [20]:
proj = context_length * d_model * d_model * 4 * 2 # 2 flops per multily-add op

print(proj * num_layers)

1006632960000



Attention: Q @ K & softmax @ V

In [ ]:
attn_qk = context_length * context_length * d_model * 2

# (... context_lenght context_length) @ (context_length d_model)
attn_v = context_length * d_model * context_length * 2

attn = attn_qk + attn_v

print((attn + proj) * num_layers)


322122547200
1328755507200



RoPE: None


3. FFN (SwiGLU): w1x, w2x, w3x


In [17]:
ffn = context_length * d_ff * d_model * 3 * 2

print(ffn * num_layers)

3019898880000



### Output embedding

In [ ]:
output_linear = context_length * d_model * vocab_size * 2

### Total Flops


In [22]:
mhsa_flops = proj + attn
transformer_block_flops = mhsa_flops + ffn
total_flops = num_layers * transformer_block_flops + output_linear

print(total_flops)

4430995456000


## (c) Based on your analysis above, which parts of the model require the most FLOPs?


**FFN** See below

In [24]:
print(f"{ffn:,}")
print(f"{attn:,}")
print(f"{proj:,}")

62,914,560,000
6,710,886,400
20,971,520,000


## (d) Repeat your analysis with GPT-2 small (12 layers, 768 d_model, 12 heads), GPT-2 medium (24 layers, 1024 d_model, 16 heads), and GPT-2 large (36 layers, 1280 d_model, 20 heads). 

As the model size increases, which parts of the Transformer LM take up proportionally more or less of
the total FLOPs?

**Deliverable**: For each model, provide a breakdown of model components and its associated
FLOPs (as a proportion of the total FLOPs required for a forward pass). In addition, provide a
one-to-two sentence description of how varying the model size changes the proportional FLOPs
of each component.